In [ ]:
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import sklearn; print('Scikit-Learn Version:', sklearn.__version__)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)
import cudf; print("cudf Version:", cudf.__version__)


from xgboost import XGBClassifier
# from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV

In [ ]:
f = open('X_train.csv'); colNames = f.readline().strip().split(','); f.close()
cudf_X_train = cudf.read_csv('X_train.csv', delimiter=',' ,names=colNames)
cudf_X_test = cudf.read_csv('X_test.csv', delimiter=',', names=colNames)
cudf_y_train = cudf.read_csv('y_train.csv')
cudf_y_test = cudf.read_csv('y_test.csv')


In [ ]:
print(colNames)

In [ ]:
xgb.DMatrix(cudf_X_train, label=cudf_y_train)


In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=False, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(cudf_X_train, label=cudf_y_train)
        cvresult = xgb.cv(xgb_param, cudf_X_train,num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(cudf_X_train, cudf_y_train,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(cudf_X_train)
    dtrain_predprob = alg.predict_proba(cudf_X_train)[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(cudf_y_train, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(cudf_y_train, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
#Choose all predictors except target & IDcols
predictors = [x for x in cudf_X_train.columns]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, cudf_X_train, predictors)